In [1]:
import pdal
from shapely import geometry
from shapely.wkt import loads
from shapely.geometry import Polygon
import numpy as np
import traceback
import json
import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from numpy.lib import recfunctions as rfn

# import laspy
import scipy
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN, KMeans
from sklearn import metrics
from sklearn import preprocessing
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import path

In [122]:
def ept_reader(polygon_wkt: str) -> np.ndarray:
    """
    Parameters
    ----------
        Path to ept directory
    :param polygon_wkt : wkt
        WKT with clipping polygon

    Returns
    -------
    points : (Mx3) array
        The ept points
    """
    polygon = loads(polygon_wkt)
    bbox = polygon.bounds
    ept_location: str = '/var/data/entwine/ahn3/ept.json'
    ept_location: str = 'http://ngi.geodan.nl/maquette/colorized-points/ahn3_nl/ept-subsets/ept.json'
    bounds = f"([{bbox[0]},{bbox[2]}],[{bbox[1]},{bbox[3]}])"

    pipeline_config = {
        "pipeline": [
            {
                "type": "readers.ept",
                "filename": ept_location,
                "bounds": bounds
            },
            {
                "type": "filters.crop",
                "polygon": polygon_wkt
            },
            {
                "type":"filters.range",
                    "limits":"NumberOfReturns[3:]"
            },
            {
                "type":"filters.smrf"
            },
            {
                "type":"filters.range",
                "limits":"Classification[:1]"
            },

        ]
    }


    try:
        # Currently the filter is not limited by numberofreturns or normalz
        pipeline = pdal.Pipeline(json.dumps(pipeline_config))
        pipeline.validate()  # check if our JSON and options were good
        pipeline.execute()
    except Exception as e:
        trace = traceback.format_exc()
        print("Unexpected error:", trace)
        print('Polygon:', polygon_wkt)
        print("Error:", e)
        raise
    
    arrays = pipeline.arrays
    points = arrays[0]
    return points


def write_to_laz(point_cloud, path):
    '''
    writes a structured array to a .laz file
    in:
        point_cloud [structured np array]:
            The output pointcloud; needs attributes x, y and z. 
            When createing a pointcloud from scratch, pay attention to 
            the data types of the specific attributes, this is a pain in the ass.
            Easier to add one new collumn to an existing (filtered) pointcloud.
        path [string]:
            Path to a laz file.
    out:
        None
    '''
    WRITE_PIPELINE = """
    {{
        "pipeline": [
            {{
                "type": "writers.las",
                "filename": "{path}",
                "extra_dims": "all"
            }}
        ]
    }}
    """
    pipeline = pdal.Pipeline(
        WRITE_PIPELINE.format(path=path),
        arrays=[point_cloud]
    )
    pipeline.validate()
    pipeline.execute()

In [123]:
from sklearn.preprocessing import StandardScaler
from hdbscan import HDBSCAN

# seperate trees
# minx, miny, maxx, maxy = 122826.8, 490054.2, 122944.5, 490134.3

# separate and dense
# minx, miny, maxx, maxy = 122504.4,490066.9, 122665.8,490160.5

minx, miny, maxx, maxy = 122619.6,490344.5, 122668.2,490373.4


wkt_box = geometry.box(minx, miny, maxx, maxy).wkt
print(wkt_box)

pts = ept_reader(wkt_box)

POLYGON ((122668.2 490344.5, 122668.2 490373.4, 122619.6 490373.4, 122619.6 490344.5, 122668.2 490344.5))


In [124]:
f_pts = pd.DataFrame(pts[['X','Y','Z','Red','Green','Blue','Intensity','ReturnNumber']])

data = f_pts.drop(['X','Y','Z'], axis = 1)

scaler = StandardScaler()
scaler.fit(data)
st_data = pd.DataFrame(scaler.transform(data), columns = data.columns)

st_data = f_pts[['X','Y','Z']].join(st_data)
st_data['pid'] = st_data.index

# first cluster on XY

In [125]:
xy = np.array([st_data.X, st_data.Y]).T
xy_clusterer = HDBSCAN(min_cluster_size=40, 
                        min_samples = 60)
xy_clusterer.fit(xy)
st_data['xy_clusterID'] = xy_clusterer.labels_

# second cluster on color 

In [137]:
from sklearn.cluster import KMeans,MeanShift
from sklearn.metrics import silhouette_score

sil = []
kmax = 10

def kmean_cluster(min_clusts, max_clusts, data):
    opt_k = 0
    p_score = 0
    largest_diff = 0
    # dissimilarity would not be defined for a single cluster, thus, minimum number of clusters should be 2
    for k in range(min_clusts+1, max_clusts+1):
        kmeans = KMeans(n_clusters = k).fit(data)
        labels = kmeans.labels_
        n_score = silhouette_score(data, labels, metric = 'euclidean')
        diff = abs(n_score - p_score )
        print(diff)
        if diff > largest_diff:
            largest_diff = diff
            opt_k = k
        p_score = n_score
    
    

In [138]:
labs = np.array([])
n_ids = np.array([])
# for name, group in st_data.groupby('xy_clusterID'):
#     if group.shape[0] >= 2000:
#         value_clusterer = HDBSCAN(min_cluster_size=40, min_samples = 10)
#         value_clusterer.fit(group.loc[:,['X','Y','Z']].T)
#         labs = np.append(labs, value_clusterer.labels_)
#     else:
#         labs = np.append(labs, [1] * group.shape[0])
        

for name, group in st_data.groupby('xy_clusterID'):
    if group.shape[0] >= 3000:
        k_labs = kmean_cluster(1,10, group[['X','Y','Z']])
        labs = np.append(labs, k_labs)
    else:
        labs = np.append(labs, [1] * group.shape[0])
    n_ids = np.append(n_ids, group.pid)

arr_inds = n_ids.argsort()
sorted_labs = labs[arr_inds]
st_data['value_clusterID'] = sorted_labs
mask = np.logical_and(st_data.xy_clusterID >=0 , st_data.value_clusterID >= 0)
combi_ids = ["".join(row) for row in st_data[['value_clusterID', 'xy_clusterID']].values.astype(str)]
st_data['unique_CID'] = pd.factorize(combi_ids)[0]

print(f'there are {f_pts.shape[0]} pts in the img')
print(f'there are {len(set(st_data.xy_clusterID))} xyclusters')
print(f'there are {len(set(st_data.value_clusterID))} value clusters')
print(f'there are {len(set(st_data.unique_CID))} unique clusters')
print(f'there are {list(mask).count(False)} noise, which is {round(list(mask).count(False)/f_pts.shape[0],2)} procenten')

0.520548930347634
0.1267680571672996
0.039266048139865284
0.03228438229167574
0.025355096775985464
0.021980183437920076
0.007384818832738671
0.013210815608755477
0.007827745277809106


IndexError: index 6778 is out of bounds for axis 0 with size 1258

# write to file

In [128]:
out_pts = pts[['X','Y','Z','Red','Green','Blue','Intensity','ReturnNumber']]
n_pts = rfn.append_fields(out_pts[mask], 'Classification', st_data[mask]['unique_CID'])
write_to_laz(n_pts, 'filtered_pts.laz')

In [ ]:
f_pts[mask]

In [ ]:
wkt = 'MULTIPOINT ('
for lab in set(lab_pts["ClusterID"]):
    tre = n_pts[n_pts['Classification'] == lab]
    if tre.shape[0] >= 50:
        tre_x = tre['X'].mean()
        tre_y = tre['Y'].mean()
        wkt += f'{tre_x} {tre_y}, '

wkt = wkt[:-2] + ')'
print(wkt)
    

In [87]:
from sklearn.cluster import KMeans



In [94]:
group[]

,X,Y,Z,Red,Green,Blue,Intensity,ReturnNumber,xy_clusterID,value_clusterID,unique_CID,pid
46,122622.40,490109.44,14.54,1.434361,1.835011,1.771336,0.209410,-1.200406,15,0.0,9,46
93,122616.69,490109.56,12.74,-0.203316,0.023247,0.093105,-0.660342,-1.200406,15,0.0,9,93
237,122621.63,490112.55,12.08,1.049025,1.508893,1.491631,-0.003590,-1.200406,15,0.0,9,237
252,122616.95,490108.53,12.70,1.338027,1.726305,1.931168,-0.731342,-1.200406,15,0.0,9,252
253,122621.04,490107.77,2.75,1.434361,1.726305,1.731378,-0.323091,-0.230601,15,0.0,9,253
...,...,...,...,...,...,...,...,...,...,...,...,...
127076,122619.01,490112.80,1.09,0.824246,1.327717,1.251884,-0.411841,0.739205,15,1.0,20,127076
127078,122617.93,490113.16,4.30,0.149908,0.421835,0.692473,-0.323091,-0.230601,15,1.0,20,127078
127079,122617.53,490113.11,5.42,-0.331762,-0.194164,0.093105,0.777412,-1.200406,15,1.0,20,127079
127080,122619.12,490113.32,1.02,0.663689,0.929129,1.331799,-0.447342,0.739205,15,1.0,20,127080
